In [1]:
import numpy as np 
import nltk
from nltk.corpus import inaugural

In [2]:
nltk.download('brown')

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [3]:
from nltk.corpus import brown
tagged_sentences = brown.tagged_sents()
num_sentences = len(brown.tagged_sents())

In [4]:
START_SYMBOL = '^'
END_SYMBOL = '.'
print(tagged_sentences[0])

[('The', 'AT'), ('Fulton', 'NP-TL'), ('County', 'NN-TL'), ('Grand', 'JJ-TL'), ('Jury', 'NN-TL'), ('said', 'VBD'), ('Friday', 'NR'), ('an', 'AT'), ('investigation', 'NN'), ('of', 'IN'), ("Atlanta's", 'NP$'), ('recent', 'JJ'), ('primary', 'NN'), ('election', 'NN'), ('produced', 'VBD'), ('``', '``'), ('no', 'AT'), ('evidence', 'NN'), ("''", "''"), ('that', 'CS'), ('any', 'DTI'), ('irregularities', 'NNS'), ('took', 'VBD'), ('place', 'NN'), ('.', '.')]


In [5]:
sentences = []
for i in range(num_sentences):
    sentences.append(tagged_sentences[i])

In [6]:
for i in range(num_sentences):
    for j in range(len(sentences[i])):
        x = list(sentences[i][j])
        x[0] = x[0].lower()
        sentences[i].pop(j)
        sentences[i].insert(j, tuple(x))

In [7]:
for i in range(num_sentences):
    sentences[i] = [(START_SYMBOL, START_SYMBOL)] + sentences[i]

In [8]:
print(sentences[0])

[('^', '^'), ('the', 'AT'), ('fulton', 'NP-TL'), ('county', 'NN-TL'), ('grand', 'JJ-TL'), ('jury', 'NN-TL'), ('said', 'VBD'), ('friday', 'NR'), ('an', 'AT'), ('investigation', 'NN'), ('of', 'IN'), ("atlanta's", 'NP$'), ('recent', 'JJ'), ('primary', 'NN'), ('election', 'NN'), ('produced', 'VBD'), ('``', '``'), ('no', 'AT'), ('evidence', 'NN'), ("''", "''"), ('that', 'CS'), ('any', 'DTI'), ('irregularities', 'NNS'), ('took', 'VBD'), ('place', 'NN'), ('.', '.')]


In [9]:
sentence1 = sentences[0]
sentence1

[('^', '^'),
 ('the', 'AT'),
 ('fulton', 'NP-TL'),
 ('county', 'NN-TL'),
 ('grand', 'JJ-TL'),
 ('jury', 'NN-TL'),
 ('said', 'VBD'),
 ('friday', 'NR'),
 ('an', 'AT'),
 ('investigation', 'NN'),
 ('of', 'IN'),
 ("atlanta's", 'NP$'),
 ('recent', 'JJ'),
 ('primary', 'NN'),
 ('election', 'NN'),
 ('produced', 'VBD'),
 ('``', '``'),
 ('no', 'AT'),
 ('evidence', 'NN'),
 ("''", "''"),
 ('that', 'CS'),
 ('any', 'DTI'),
 ('irregularities', 'NNS'),
 ('took', 'VBD'),
 ('place', 'NN'),
 ('.', '.')]

In [10]:
## Transition Probabilities
count_tags_bigram = {}
for j in range(len(sentences)):
    for i, item in enumerate(sentences[j]):
        if(i == len(sentences[j])-1):
            continue
        
        tuple_tags = (sentences[j][i+1][1], sentences[j][i][1])
        if tuple_tags in count_tags_bigram.keys():
            count_tags_bigram[tuple_tags] += 1
            
        else:
            count_tags_bigram[tuple_tags] = 1

In [11]:
print(count_tags_bigram)

{('AT', '^'): 8297, ('NP-TL', 'AT'): 809, ('NN-TL', 'NP-TL'): 1506, ('JJ-TL', 'NN-TL'): 132, ('NN-TL', 'JJ-TL'): 1757, ('VBD', 'NN-TL'): 402, ('NR', 'VBD'): 113, ('AT', 'NR'): 35, ('NN', 'AT'): 48372, ('IN', 'NN'): 42252, ('NP$', 'IN'): 874, ('JJ', 'NP$'): 327, ('NN', 'JJ'): 28407, ('NN', 'NN'): 11646, ('VBD', 'NN'): 3306, ('``', 'VBD'): 71, ('AT', '``'): 493, ("''", 'NN'): 2797, ('CS', "''"): 52, ('DTI', 'CS'): 225, ('NNS', 'DTI'): 375, ('VBD', 'NNS'): 1421, ('NN', 'VBD'): 531, ('.', 'NN'): 19857, ('RBR', 'NN'): 116, ('VBD', 'RBR'): 47, ('IN', 'VBD'): 4381, ('NN', 'IN'): 17225, ('NNS', 'NN'): 5985, ('CS', 'NNS'): 1081, ('AT', 'CS'): 5637, ('NN-TL', 'AT'): 2564, (',', 'NN-TL'): 1617, ('WDT', ','): 778, ('HVD', 'WDT'): 144, ('JJ', 'HVD'): 57, ('AT', 'IN'): 43271, (',', 'NN'): 18279, ('``', ','): 1093, ('VBZ', '``'): 31, ('AT', 'VBZ'): 1382, ('CC', 'NN'): 9061, ('NNS', 'CC'): 2024, ('IN', 'NNS'): 14504, ('IN-TL', 'NN-TL'): 1189, ('NP-TL', 'IN-TL'): 414, ("''", 'NP-TL'): 100, ('IN', "''")

In [12]:
probability_tags_bigram = {}
total = 0
for key in count_tags_bigram.keys():
    total += count_tags_bigram[key]
    
    
for key in count_tags_bigram.keys():
    probability_tags_bigram[key] = count_tags_bigram[key]/total

In [13]:
print(probability_tags_bigram)

{('AT', '^'): 0.007145243852868432, ('NP-TL', 'AT'): 0.0006966978759757215, ('NN-TL', 'NP-TL'): 0.0012969431411859538, ('JJ-TL', 'NN-TL'): 0.00011367629125932662, ('NN-TL', 'JJ-TL'): 0.0015131003313836127, ('VBD', 'NN-TL'): 0.00034619597792613107, ('NR', 'VBD'): 9.731379479018112e-05, ('AT', 'NR'): 3.0141440864215392e-05, ('NN', 'AT'): 0.041657193642395056, ('IN', 'NN'): 0.03638674741128082, ('NP$', 'IN'): 0.0007526748375806929, ('JJ', 'NP$'): 0.000281607176074241, ('NN', 'JJ'): 0.024463654589421905, ('NN', 'NN'): 0.0100293491515615, ('VBD', 'NN'): 0.0028470743856313166, ('``', 'VBD'): 6.114406575312266e-05, ('AT', '``'): 0.0004245637241730911, ("''", 'NN'): 0.0024087317170631557, ('CS', "''"): 4.4781569283977154e-05, ('DTI', 'CS'): 0.00019376640555567037, ('NNS', 'DTI'): 0.00032294400925945066, ('VBD', 'NNS'): 0.001223742499087145, ('NN', 'VBD'): 0.0004572887171113821, ('.', 'NN'): 0.01710053117830643, ('RBR', 'NN'): 9.989734686425673e-05, ('VBD', 'RBR'): 4.0475649160517815e-05, ('IN'

In [14]:
## Lexical Independence Probabilities / Emission Probability
emission_probabilities_dict = {'.' : {'.' : 1}}

for j in range(len(sentences)):
    for i, item in enumerate(sentences[j]):
        if(i == len(sentences[j])-1):
            continue
        
        if(sentences[j][i][1] in emission_probabilities_dict):
            if(sentences[j][i][0] in emission_probabilities_dict[sentences[j][i][1]]):
                emission_probabilities_dict[sentences[j][i][1]][sentences[j][i][0]] += 1
                
            else:
                emission_probabilities_dict[sentences[j][i][1]][sentences[j][i][0]] = 1
                
        else:
            emission_probabilities_dict[sentences[j][i][1]] = {}
            emission_probabilities_dict[sentences[j][i][1]][sentences[j][i][0]] = 1

In [15]:
tags = list(emission_probabilities_dict.keys())

In [16]:
for key1 in emission_probabilities_dict.keys():
    total = 0
    for key2 in emission_probabilities_dict[key1].keys():
        total += emission_probabilities_dict[key1][key2]
    
    for key3 in emission_probabilities_dict[key1].keys():
        emission_probabilities_dict[key1][key3] /= total

In [17]:
emission_probabilities_dict

{'.': {'.': 0.00016909029421711193,
  '?': 0.39127494081839703,
  ';': 0.4702401082177883,
  '!': 0.13442678390260399,
  ':': 0.0038890767669935745},
 '^': {'^': 1.0},
 'AT': {'the': 0.7045178416458217,
  'an': 0.037837561896983,
  'no': 0.01842870999030068,
  'a': 0.23419265914543877,
  'every': 0.005002807698197968,
  "th'": 1.0209811628975445e-05,
  "ever'": 1.0209811628975445e-05},
 'NP-TL': {'fulton': 0.0024906600249066002,
  'atlanta': 0.00099626400996264,
  'grady': 0.00099626400996264,
  'georgia': 0.0034869240348692405,
  'jackson': 0.00049813200498132,
  'miller': 0.00049813200498132,
  'colquitt': 0.00024906600249066,
  'texas': 0.004234122042341221,
  'dallas': 0.0032378580323785804,
  'beaumont': 0.00024906600249066,
  'lamar': 0.00049813200498132,
  'texan': 0.00024906600249066,
  'wise': 0.00024906600249066,
  'paris': 0.00099626400996264,
  'oklahoma': 0.0007471980074719801,
  'rhode': 0.024906600249066,
  'massachusetts': 0.0034869240348692405,
  'u.s.': 0.014196762141

In [18]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [19]:
from nltk.tokenize import word_tokenize
input_sent = "reading is an essential skill."
input_sent_tokenized = word_tokenize(input_sent)

input_sent_tokenized

['reading', 'is', 'an', 'essential', 'skill', '.']

In [20]:
viterbi_matrix = []
ls = []
for tag in tags:
    if((tag, '^') in probability_tags_bigram.keys()):
        if(input_sent_tokenized[0] in emission_probabilities_dict[tag].keys()):
            ls.append(('^', probability_tags_bigram[(tag, '^')] * emission_probabilities_dict[tag][input_sent_tokenized[0]]))
        
        else:
            ls.append(('^', 0))
    
    else:
        ls.append(('^', 0))
        
viterbi_matrix.append(ls)


In [21]:
print(viterbi_matrix)

[[('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 4.046155621582167e-07), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 9.366835150599618e-08), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 2.3975686696820562e-06), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 3.5669367082156004e-07), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 

In [22]:
tags = list(tags)
# tags.pop(0)
# tags.pop(0)
tags

['.',
 '^',
 'AT',
 'NP-TL',
 'NN-TL',
 'JJ-TL',
 'VBD',
 'NR',
 'NN',
 'IN',
 'NP$',
 'JJ',
 '``',
 "''",
 'CS',
 'DTI',
 'NNS',
 'RBR',
 ',',
 'WDT',
 'HVD',
 'VBZ',
 'CC',
 'IN-TL',
 'BEDZ',
 'VBN',
 'NP',
 'BEN',
 'TO',
 'VB',
 'RB',
 'DT',
 'PPS',
 'DOD',
 'AP',
 'BER',
 'HV',
 'DTS',
 'VBG',
 'PPO',
 'QL',
 'JJT',
 'ABX',
 'NN-HL',
 'WRB',
 'CD',
 'MD',
 'BE',
 'JJR',
 'VBG-TL',
 'BEZ',
 'NN$-TL',
 'HVZ',
 'ABN',
 'PN',
 'PPSS',
 'PP$',
 'DO',
 'NN$',
 'NNS-HL',
 'WPS',
 '*',
 'EX',
 'VB-HL',
 ':',
 '(',
 'NNS-TL',
 'NPS',
 'JJS',
 'RP',
 '--',
 'BED',
 'OD',
 'BEG',
 'AT-HL',
 'VBG-HL',
 'AT-TL',
 'PPL',
 ')',
 'DOZ',
 'NR$',
 'DOD*',
 'BEDZ*',
 'NP-HL',
 ',-HL',
 'CC-TL',
 'MD*',
 'NNS$',
 'PPSS+BER',
 "'",
 'PPSS+BEM',
 'CD-TL',
 'RBT',
 '(-HL',
 'MD-HL',
 'VBZ-HL',
 'IN-HL',
 'JJ-HL',
 'PPLS',
 'WPO',
 'JJS-TL',
 'ABL',
 'BER-HL',
 'PPS+HVZ',
 'VBD-HL',
 'CD-HL',
 'RP-HL',
 'MD*-HL',
 'AP-HL',
 'CS-HL',
 'DT$',
 'HVN',
 'FW-IN',
 'FW-DT',
 'VBN-TL',
 'NR-TL',
 'NNS$-TL',
 'FW

In [23]:
## Each layer maps to 1 tag initially, the max probability from these layers to the corresponding tag is continued as a path, the rest are pruned.
## The viterbi matrix will append a list of tuples, each tuple has the following form: (parent/prev, max_probability) 

In [24]:
for i in range(1, len(input_sent_tokenized)):
    ls = [] 
    for tag in tags:
        max_probability = 0
        max_tag = '$Can$'
        for j in range(len(viterbi_matrix[i-1])):
            if(viterbi_matrix[i-1][j][1] > 0):
                prev_tag = tags[j]
                prev_probability = viterbi_matrix[i-1][j][1]
                
                if (tag, prev_tag) in probability_tags_bigram.keys() and input_sent_tokenized[i] in emission_probabilities_dict[tag].keys():
                    total_probability  = prev_probability * probability_tags_bigram[(tag, prev_tag)] * emission_probabilities_dict[tag][input_sent_tokenized[i]]
                    if(total_probability > max_probability):
                        max_probability = total_probability
                        max_tag = prev_tag            
        
        ls.append((max_tag, max_probability))                
    viterbi_matrix.append(ls)  
            

In [25]:
for item in viterbi_matrix:
    print(item)

[('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 4.046155621582167e-07), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 9.366835150599618e-08), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 2.3975686696820562e-06), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 3.5669367082156004e-07), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0), ('^', 0

In [32]:
for i,item in enumerate(viterbi_matrix):
    max_val = -1
    max_tag = ''
    for x in item:
        if(x[1] > max_val):
            max_val = x[1]
            max_tag = x[0]
    if(i == 0): print('Start: ' + max_tag)
    else: print(input_sent_tokenized[i-1] + ": " + max_tag)


Start: ^
reading: NN
is: BEZ
an: AT
essential: JJ
skill: NN
